### Describing most common emotions from given messenger conversations
The program uses two models to present the most popular emotions which can be detected in conversations.

First we have to insert in the first line the name of the json file with the conversation. The instruction how to download such files from facebook is in README. Then just run all the cells step by step. Have fun!

Code below fixes polish signs in json like 'ę', 'ą' etc and saves result in a new file

In [1]:
input_file_path = 'piotrek.json'  # Podmień nazwę pliku wejściowego
#pewnie nieoptymalnie ale działa XD

def fix_polish_characters(input_file, output_file):
    with open(input_file, 'rb') as file:
        json_data = file.read().decode('utf-8')
        decoded_json = json_data.replace(r'\u00c4\u0099', 'ę').replace(r'\u00c5\u009b','ś').replace(r'\u00c5\u0082','ł')\
            .replace(r'\u00c4\u0087','ć').replace(r'\u00c5\u00bc','ż').replace(r'\u00c4\u0085','ą').replace(r'\u00c3\u00b3','ó')\
            .replace(r'\u00c5\u0084','ń')
        
        with open(output_file, 'w', encoding='utf-8') as output_file:
            output_file.write(decoded_json)

# Przykład użycia
output_file_path = "fixed_"+input_file_path  # Podmień nazwę pliku wyjściowego

fix_polish_characters(input_file_path, output_file_path)


Code below reads messages from json and saves then in txt file

In [2]:
import json

# Wczytaj plik JSON
with open(output_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

    
new_file_name = "data_" + output_file_path.replace(".json", ".txt")

# Otwórz nowy plik tekstowy do zapisu
with open(new_file_name, 'w', encoding='utf-8') as output_file:
    # Iteruj przez wiadomości i zapisuj zawartość do pliku
    for message in data['messages']:
        if 'content' in message:
            output_file.write(message['content'] + '. ')


Code below uses the first model to translate sentences from polish to english, because the next model needs english language

In [3]:
from transformers import MarianMTModel, MarianTokenizer

def translate_polish_to_english(text):
    # Ładowanie modelu i tokenizera dla tłumaczenia z polskiego na angielski
    model_name = "Helsinki-NLP/opus-mt-pl-en"
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)

    # Tokenizacja tekstu
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

    # Tłumaczenie
    translation = model.generate(**inputs)
    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)

    return translated_text


with open(new_file_name, 'r',encoding='utf-8') as file:
    file_content = file.read()

# Przykład użycia
# polish_text = "Przykładowy tekst do przetłumaczenia."
english_translation = translate_polish_to_english(file_content)
print("Tekst po angielsku:", english_translation)


Tekst po angielsku: Fortunately, you're replying quickly. Okay. He didn't reply to me. Okay. Okay. I don't know what it is for XD courier. Okay. I'll be calling. I'll go. I'll go. I'll go. If you want to go like that. Okay. I'm sure. I'm sure. I'm sure. I'm sure. I'm sure. I'm sure. I'll go for it. I'll go for it. I'll go for it. I'll go for it. I'll go for it. I'm sorry. I'm sure. I'm sure. I'm sure. I'm sure.


Code below uses the second model to find emotions in the conversation and present them as a result

In [4]:
from transformers import pipeline

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)
    
sentences = [english_translation]

model_outputs = classifier(sentences)
print(model_outputs[0])
# produces a list of dicts for each of the labels


[{'label': 'approval', 'score': 0.8042270541191101}, {'label': 'caring', 'score': 0.07204558700323105}, {'label': 'optimism', 'score': 0.06316885352134705}, {'label': 'gratitude', 'score': 0.043394383043050766}, {'label': 'realization', 'score': 0.042865291237831116}, {'label': 'relief', 'score': 0.042029306292533875}, {'label': 'neutral', 'score': 0.04092206060886383}, {'label': 'remorse', 'score': 0.025647573173046112}, {'label': 'joy', 'score': 0.025023262947797775}, {'label': 'confusion', 'score': 0.024801533669233322}, {'label': 'disapproval', 'score': 0.024592392146587372}, {'label': 'annoyance', 'score': 0.013317607343196869}, {'label': 'admiration', 'score': 0.01140284352004528}, {'label': 'disappointment', 'score': 0.010581821203231812}, {'label': 'sadness', 'score': 0.010523905977606773}, {'label': 'nervousness', 'score': 0.005780561361461878}, {'label': 'pride', 'score': 0.004807025659829378}, {'label': 'curiosity', 'score': 0.0045570870861411095}, {'label': 'embarrassment',